# RANS from DNS

Our goal here, in general, is to find new valid equations that describe fluid flow. 
We will try to find new closures for the steady RANS equations based on direct numerical
simulation (DNS) of a boundary layer.


## Steady RANS equations with generic Reynolds stress effects

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= \mathbf{R},
$$

where in this case $\mathbf{R}$ is simply the effects of the Reynolds stresses (i.e., the opposite of the gradient), not the Reynolds stresses themselves.

Some ideas for what $\mathbf{R}$ could be:

$$
\mathbf{R} = A\nabla P^2 + B\nabla K + C \nabla \times \vec{U} 
    + D\nabla(\nabla \times \vec{U})^2
    + E \vec{U}
$$


## Algorithm

1. Pick terms (in addition to non-Reynolds stress Navier--Stokes terms).
2. Create a random list of points in space that is at least as large as the number
   of terms.
3. At each point, acquire all data for all terms for all times.
4. Average data at each point for all times.
5. Solve for coefficients using a linear model.

## Terms

$$
U \frac{\partial U}{\partial x} 
+ V \frac{\partial U}{\partial y} + W \frac{\partial U}{\partial z}
+ \frac{1}{\rho}\frac{\partial P}{\partial x} 
- \nu \left( 
    \frac{\partial^2 U}{\partial x^2}
    + \frac{\partial^2 U}{\partial y^2} 
    + \frac{\partial^2 U}{\partial z^2} 
\right)
$$

$$
= 
A \left( \frac{\partial U}{\partial x} \right)^2 
+ B \left( \frac{\partial U}{\partial y} \right)^2
+ C \left( \frac{\partial U}{\partial z} \right)^2
+ D \left( \frac{\partial P}{\partial x} \right)^2
+ E \frac{\partial^2 P}{\partial x^2}
+ F U \frac{\partial P}{\partial x}
$$

$$
U \frac{\partial V}{\partial x} 
+ V \frac{\partial V}{\partial y} 
+ W \frac{\partial V}{\partial z}
+ \frac{1}{\rho}\frac{\partial P}{\partial y} 
- \nu \left( 
    \frac{\partial^2 V}{\partial x^2}
    + \frac{\partial^2 V}{\partial y^2} 
    + \frac{\partial^2 V}{\partial z^2} 
\right)
$$

$$
= 
A \left( \frac{\partial V}{\partial x} \right)^2 
+ B \left( \frac{\partial V}{\partial y} \right)^2
+ C \left( \frac{\partial V}{\partial z} \right)^2
+ D \left( \frac{\partial P}{\partial y} \right)^2
+ E \frac{\partial^2 P}{\partial y^2}
+ F V \frac{\partial P}{\partial y}
$$

## Terms in index notation

To be general and consistent, since we don't have any x- or z-variation

$$
\frac{\partial U_i}{\partial t} + U_j \frac{\partial U_i}{\partial x_j} 
+ \frac{1}{\rho}\frac{\partial P}{\partial x_i}
- \nu \frac{\partial ^2 U_i}{\partial x_j x_j}
=
A \frac{(\partial U_i)^2}{\partial x_j \partial x_j}
+ B \frac{\partial U_j U_j}{\partial x_i}
+ C \frac{\partial P^2}{\partial x_i}
+ D \left( \frac{\partial P}{\partial x_i} \right)^2
+ E U_j \frac{\partial P}{\partial x_j}
$$

What these coefficients describe:

* $A$: The square of the velocity gradient
* $B$: The gradient of kinetic energy
* $C$: The gradient of squared pressure
* $D$: The square of the pressure gradient

In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn
seaborn.set()

## Using data from pyJHTDB

1. Pick a bunch of points randomly throughout the domain, at least more than the number of terms we want to test.
2. Add points in each direction for computing spatial derivatives.
3. Get $\vec{u}$, $p$, and their gradients for all time at all points in the list.
4. Calculate terms based on mean values.
5. Use a regression model to determine coefficients on each term.
6. Repeat this process to ensure the coefficients don't change?
7. Run a RANS simulation with this new model and check the results against the mean profiles.

In [ ]:
with h5py.File("data/jhtdb-transitional-bl/time-ave-profiles.h5", "r") as f:
    print(f.keys())
    data = {}
    for k in f.keys():
        kn = k.split("_")[0]
        if kn.endswith("m"):
            kn = kn[:-1]
        data[kn] = f[k][()]

In [ ]:
# If numerical derivatives are too noisy, download gradients and Hessians from
# JHTDB and time average
dx = np.gradient(data["x"])
dy = np.reshape(np.gradient(data["y"]), (224, 1))
# dz = np.gradient(data["z"])
# TODO: Correct fluctuation terms according to README
# >uum is the time-averaged of u*u (not u'*u', where u'=u-um).
# >So time-averaged of u'*u'=uum-um*um. Same for other quantities.
for dim in ("u", "v", "w"):
    data[f"{dim}{dim}"] = data[f"{dim}{dim}"] - data[f"{dim}"]**2
# Calculate gradients
data["dpdx"] = np.gradient(data["p"], axis=1) / dx
data["duudx"] = np.gradient(data["uu"], axis=1) / dx
data["duvdy"] = np.gradient(data["uv"], axis=0) / dy
data["dudx"] = np.gradient(data["u"], axis=1) / dx
data["dudy"] = np.gradient(data["u"], axis=0) / dy
data["d2udx2"] = np.gradient(data["dudx"], axis=1) / dx
data["d2udy2"] = np.gradient(data["dudy"], axis=0) / dy
data["dpdx"] = np.gradient(data["p"], axis=1) / dx
data["dpdy"] = np.gradient(data["p"], axis=0) / dy
# data["dwdz"] = np.gradient(data["w"], axis=1) / dz
pd.Series(
    {k: v.shape for k, v in data.items()}
)

In [ ]:
plt.plot(data["dpdx"][100, :], data["x"])

In [ ]:
plt.plot(data["duudx"][:, 100], data["y"])

In [ ]:
# TODO: Check that RANS applies to these mean values
# First, let's check the momentum balance in the x-direction
rho = 1
nu = 1.25e-3
momx = (
    data["u"] * data["dudx"]
    + data["v"] * data["dudy"]
    # + data["w"] * data["dudz"]  # 2-D; zero
    + rho * data["dpdx"]
    - nu * (data["d2udx2"] + data["d2udy2"])
    + rho * (data["duudx"] + data["duvdy"])
    # + data["duwdz"]  # 2-D
)
momx

In [ ]:
plt.plot(momx[:, 100], data["y"])

In [ ]:
# Let's check continuity
div = data["dudx"] + data["duvdy"]
div

The check above gives us an idea on how accurate these gradient calculations
are.

In [ ]:
# TODO: Compute a bunch of quantities and add to the data dictionary
# Mean kinetic energy
# Squared gradients
# Gradients multiplied by each other
# Gradients multiplied by mean values

In [ ]:
# Compute the Reynolds stress residual as a target for an ML model
# Solve a linear regression for the coefficients of all derived terms
# Throw out terms with coefficients below a threshold

In [ ]:
# TODO: Write as a RANS model for OpenFOAM and solve this same problem there

In [ ]:
# TODO: Check mean flow from OpenFOAM simulation matches DNS